In [1]:
import pandas as pd

song_data = pd.read_csv('song_data_processed.csv')
eurovision = pd.read_csv('eurovision_processed.csv')
finals = pd.read_csv('finals_processed.csv')
ogae_polls = pd.read_csv('ogae_polls.csv')
world_polls = pd.read_csv('eurovisionworld_polls.csv')
points2016 = pd.read_csv('points_2016.csv')
points2017 = pd.read_csv('points_2017.csv')
points2018 = pd.read_csv('points_2018.csv')
points2019 = pd.read_csv('points_2019.csv')
points2021 = pd.read_csv('points_2021.csv')
points2022 = pd.read_csv('points_2022.csv')

In [2]:
import plotly.express as px
import pandas as pd

# Load data
song_data = pd.read_csv("song_data_processed.csv")

# Filter data for 2022
df = song_data.loc[song_data['year'] == 2022]

# Group data by gender and count the number of occurrences
gender_counts = df['gender'].value_counts().reset_index()

# Rename columns for clarity
gender_counts.columns = ['gender', 'count']

# Create pie chart
fig = px.pie(gender_counts, values='count', names='gender', title="Gender Distribution of Songs in 2022")

# Show plot
fig.show()

fig.write_html("test.html")